In [8]:
# def remove_headers():
#     for i in range(1,41):
#         for j in range(1,11):
#             with open(f"test/data/orl_faces/s{i}/{j}.pgm", "rb") as f:
#                 lines = f.readlines()
            
#             with open(f"test/data/processed_orl_faces/s{i}/{j}.pgm", "wb") as processed_f:
#                 processed_f.writelines(lines[3:])


In [16]:
from compression.helpers import read_data
import numpy as np
from time import time

In [7]:
def get_images():
    images = []
    for i in range(1,41):
        person_images = []
        for j in range(1,11):
            person_images.append(read_data(f"test/data/orl_faces/s{i}/{j}.pgm")[0])
        images.append(person_images)
    return images

In [8]:
images = get_images()

In [9]:
len(images)

40

In [10]:
np.random.choice(10)

2

## Cross Validation

In [11]:
def cross_validation(images):
    training_images = []
    test_images = []

    for i in range(0,len(images)):
        test_index = np.random.choice(10)
        training_images.append(images[i].pop(test_index))
        test_images.append(images[i])
    return training_images, test_images

## Training

In [12]:
from compression.lzw import LZW


def train(images, dicts, k):
    lzw = LZW(dictionary_k=k)
    data_set = {i for i in range(2**8)}
    for i, person_images in enumerate(images):
        for image in person_images:
                _, dicts[i], _ = lzw.encode(image, data_set.copy(), existing_dict=dicts[i], incremental=True)

    return dicts

## Testing

In [13]:
def test(test_images, dicts, k, verbose: bool) -> float:
    lzw = LZW(dictionary_k=k)
    data_set = {i for i in range(2**8)}
    accuracy_result = []
    
    for i, image in enumerate(test_images):
        compression_size = []

        for j in range(40):
            output, _ = lzw.encode(image, data_set, existing_dict=dicts[j], incremental=False) # needs required arguments
            compression_size.append(len(output))

        accuracy_result.append(i  == compression_size.index(min(compression_size)))
    
    accuracy = 100 * accuracy_result.count(True) / len(accuracy_result)
    if verbose:
        print(f"Modelo com acurácia para k = {k}: {accuracy}%")
    return accuracy 

## Main function

In [14]:
def train_test(images, k):
    train_start = time()
    dicts = []
    data_set = {i for i in range(2**8)}
    dictionary = {(value,): i for i, value in enumerate(data_set)}

    for i in range(40):
        dicts.append(dictionary.copy())

    training_images, test_images = cross_validation(images)

    dicts = train(training_images, dicts, k)

    return test(test_images, dicts, k, verbose=True), time() - train_start

## Running

In [15]:
images = get_images()
acc = []
time = []

for k in range(9,17):
    accuracy, training_time = train_test(images, k)
    acc.append(accuracy)
    time.append(training_time)

Starting dictionary: {(0,): 0, (1,): 1, (2,): 2, (3,): 3, (4,): 4, (5,): 5, (6,): 6, (7,): 7, (8,): 8, (9,): 9, (10,): 10, (11,): 11, (12,): 12, (13,): 13, (14,): 14, (15,): 15, (16,): 16, (17,): 17, (18,): 18, (19,): 19, (20,): 20, (21,): 21, (22,): 22, (23,): 23, (24,): 24, (25,): 25, (26,): 26, (27,): 27, (28,): 28, (29,): 29, (30,): 30, (31,): 31, (32,): 32, (33,): 33, (34,): 34, (35,): 35, (36,): 36, (37,): 37, (38,): 38, (39,): 39, (40,): 40, (41,): 41, (42,): 42, (43,): 43, (44,): 44, (45,): 45, (46,): 46, (47,): 47, (48,): 48, (49,): 49, (50,): 50, (51,): 51, (52,): 52, (53,): 53, (54,): 54, (55,): 55, (56,): 56, (57,): 57, (58,): 58, (59,): 59, (60,): 60, (61,): 61, (62,): 62, (63,): 63, (64,): 64, (65,): 65, (66,): 66, (67,): 67, (68,): 68, (69,): 69, (70,): 70, (71,): 71, (72,): 72, (73,): 73, (74,): 74, (75,): 75, (76,): 76, (77,): 77, (78,): 78, (79,): 79, (80,): 80, (81,): 81, (82,): 82, (83,): 83, (84,): 84, (85,): 85, (86,): 86, (87,): 87, (88,): 88, (89,): 89, (90,): 9

TypeError: 'int' object is not subscriptable